# Scraping
Upphaflega fór skröpun fram í þessu skjali en ákvað að splitta því upp svo hægt væri að keyra alla gpt-þjálfunina á collab án þess að þurfa keyra allt draslið aftur

In [ ]:
%pip install datasets
%pip install tokenizers
%pip install transformers
%pip install seqeval
%pip install accelerate==0.24.1
%pip install wandb
# from google.colab import drive
# drive.mount('/content/drive')

# GPT-2 þjálfun

## Formöttum gögn

In [1]:
import json
from sklearn.model_selection import train_test_split

# with open('/content/drive/MyDrive/scraped_data.json') as f:
with open('scraping/scraped_data.json') as f:
    data = json.load(f)

lyrics = list(data.values())
for i in range(len(lyrics)):
    lyrics[i] = "\n".join(lyrics[i])

def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        data += texts + "  "
    f.write(data)

train, test = train_test_split(lyrics,test_size=0.15)

build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 3332
Test dataset length: 588


## Hlöðum inn tóka

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("jonfd/gpt2-igc-is")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

## Gerum gögnin "þjálfanleg"

In [2]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.11/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (118490 > 512). Running this sequence through the model will result in indexing errors


## Stillum þjálfarann

In [5]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("jonfd/gpt2-igc-is")

training_args = TrainingArguments(
    output_dir="./gpt2-textar", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=5, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Þjálfum

In [ ]:
trainer.train()

# Vistum

In [ ]:
from transformers import AutoModel
model = AutoModel.from_pretrained("gpt2-textar")
trainer.push_to_hub("ofurtumi/gpt2textar_v0.1", private=True)

In [4]:
from transformers import pipeline

gen1 = pipeline("text-generation", model="ofurtumi/gpt2textar_v0.1", tokenizer="jonfd/gpt2-igc-is", pad_token_id=50256)
gen2 = pipeline("text-generation", model="ofurtumi/gpt2textar_v0.2", tokenizer="jonfd/gpt2-igc-is", pad_token_id=50256)
gen3 = pipeline("text-generation", model="ofurtumi/gpt2textar_v0.3", tokenizer="jonfd/gpt2-igc-is", pad_token_id=50256)

versions = {"v0.1": gen1, "v0.2": gen2, "v0.3": gen3}
# gen03 = pipeline("text-generation", model="ofurtumi/gpt2textar_v0.1", tokenizer="jonfd/gpt2-igc-is")

In [5]:
def compare_versions(prompt):
    for name, version in versions.items():
        print(name)
        print(version(prompt, do_sample=True, temperature=0.9)[0]["generated_text"])
        print()

In [6]:
compare_versions("Kisuvæl")

v0.1
Kisuvæl um landið,
og um heiminn ég ætla ekki að fljúga.
Því þær eru svo mikið fjör.
Láta þær slá í sig endalaust,
alla daga sem kvöld.  Lengi getur maður verið furðufugl,

v0.2
Kisuvæl
En gamla fólkið það segir:
Ja, ja, hún er nú aldeilis angrí
og þessi lambi hún Stína,
og þessi lambi er nú hér,
nú er ekki um það neitt neitt að ræða

v0.3
Kisuvæl og læti
Hettudúfa út á túni,
hæ, hann sagði ei neitt.
Óli skans og Snæfinnur snjókarl
kisa út í móa,
kisa hljóp í átt til mín



In [7]:
compare_versions("Fólk sem hefur alltof gaman")

v0.1
Fólk sem hefur alltof gaman
kann ekkert um lífið að draga.
Ég á von á þér ein,
þú ert fyrsta vonarstjarna mín.
Ég var áður eitt, en er kominn til að vera.
Ég elska þig eins

v0.2
Fólk sem hefur alltof gaman
en samt er það bara hali
að hafa það kótilettu með rauðkorg er það sem ég hef
Í sumar ætla ég að skemmta mér
og hafa gaman af
ég veit samt ekki nema ég

v0.3
Fólk sem hefur alltof gaman.
Sorrí með mig, sorrí með mig, sorrí með mig, sorrí með mig, sorrí með mig, sorrí með mig.
Ég elska þig og þú elskar mig,
þú



In [8]:
compare_versions("Texti um barn sem týndi boltanum sínum")

v0.1
Texti um barn sem týndi boltanum sínum
Og sem var það sem þau voru ekki
Ég sá þennan dreng sem spilaði með ÍA
Við gætum haft það svo frábært að koma þér ekki fyrir kattarnef
en þá værum við bara vitlaus
Og

v0.2
Texti um barn sem týndi boltanum sínum
Áttum okkur augnablik
Í lífi sem er eitt augnablik
Já ég á mér draum sem er að rætast
Ég finn mig alveg heilan og hress
Ég á mér draum sem er að rætast
Ég á

v0.3
Texti um barn sem týndi boltanum sínum
Við gleymum öllu
og hvað það var sem var
Hvað það var sem var
Hvað það var sem var
Allt sem við hugsuðum í gær,
hvað það var sem var
Hvað það var

